# **Criando uma base de dados**

In [1]:
import pandas as pd
import numpy as np

treino= pd.read_csv("/content/train.csv")

treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#**Arrumando os dados**

**Excluindo Colunas**

In [2]:
exclui_coluna= ["Name", "Ticket", "Cabin", "Embarked", "Fare", "PassengerId"]
treino.drop(columns= exclui_coluna, inplace= True)

treino.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,male,22.0,1,0
1,1,1,female,38.0,1,0
2,1,3,female,26.0,0,0
3,1,1,female,35.0,1,0
4,0,3,male,35.0,0,0


**Transformando categorico em discreto**

In [3]:
treino["Sex"]= treino["Sex"].replace(["male", "female"], [0, 1])
treino["Sex"].head()

0    0
1    1
2    1
3    1
4    0
Name: Sex, dtype: int64

**Trantando Numeros Nulos**

In [4]:
treino.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
dtype: int64

In [5]:
# Age
def idade_mediana(df):
  mediana= df["Age"].median()
  df.update(df["Age"].fillna(mediana))
  return(df)

treino= idade_mediana(treino)
treino.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
0,0,3,0,22.0,1,0
1,1,1,1,38.0,1,0
2,1,3,1,26.0,0,0
3,1,1,1,35.0,1,0
4,0,3,0,35.0,0,0


#**Utilizando Auto-ML AutoGluon**

https://auto.gluon.ai/stable/tutorials/tabular_prediction/tabular-indepth.html

In [ ]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools
!pip install -U "mxnet<2.0.0"
# Install pre-release, frozen to a particual pre-release for stability
!pip install --pre "autogluon==0.0.16b20201214"
!pip install -U ipykernel

In [8]:
from autogluon.tabular import TabularPrediction as task

In [9]:
model= task.fit(train_data= treino, label= "Survived")

No output_directory specified. Models will be saved in: AutogluonModels/ag-20210221_012507/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20210221_012507/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    891
Train Data Columns: 5
Preprocessing data ...
AutoGluon infers your prediction problem is: 'binary' (because only two unique label-values observed).
	2 unique label values:  [0, 1]
	If 'binary' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Selected class <--> label mapping:  class 1 = 1, class 0 = 0
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12665.97 MB
	Train Data (Original)  Memory Usage: 0.04 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify 

In [10]:
model.evaluate(treino)

Predictive performance on given dataset: accuracy = 0.8653198653198653


0.8653198653198653

In [11]:
model.leaderboard(treino)

                  model  score_test  score_val  pred_time_test  pred_time_val  fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      RandomForestEntr    0.897868   0.798883        0.214353       0.103441  0.832649                 0.214353                0.103441           0.832649            0       True          2
1      RandomForestGini    0.896745   0.793296        0.113872       0.103746  0.738317                 0.113872                0.103746           0.738317            0       True          1
2        KNeighborsDist    0.894501   0.782123        0.105537       0.104477  0.004274                 0.105537                0.104477           0.004274            0       True          6
3        ExtraTreesEntr    0.890011   0.759777        0.117497       0.103386  0.516042                 0.117497                0.103386           0.516042            0       True          4
4        ExtraTreesGini    0.890011   0.75977

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestEntr,0.897868,0.798883,0.214353,0.103441,0.832649,0.214353,0.103441,0.832649,0,True,2
1,RandomForestGini,0.896745,0.793296,0.113872,0.103746,0.738317,0.113872,0.103746,0.738317,0,True,1
2,KNeighborsDist,0.894501,0.782123,0.105537,0.104477,0.004274,0.105537,0.104477,0.004274,0,True,6
3,ExtraTreesEntr,0.890011,0.759777,0.117497,0.103386,0.516042,0.117497,0.103386,0.516042,0,True,4
4,ExtraTreesGini,0.890011,0.759777,0.118750,0.103675,0.525449,0.118750,0.103675,0.525449,0,True,3
5,LightGBMCustom,0.875421,0.837989,0.013600,0.004111,0.332911,0.013600,0.004111,0.332911,0,True,12
6,XGBoost,0.865320,0.843575,0.013685,0.005028,0.343526,0.013685,0.005028,0.343526,0,True,10
7,WeightedEnsemble_L1,0.865320,0.849162,0.030525,0.010180,1.043185,0.003241,0.001041,0.366748,1,True,13
8,LightGBM,0.863075,0.826816,0.007810,0.003844,0.199066,0.007810,0.003844,0.199066,0,True,7
9,NeuralNetMXNet,0.839506,0.810056,0.049994,0.015340,4.264452,0.049994,0.015340,4.264452,0,True,11


In [12]:
model.feature_importance(treino)

Computing feature importance via permutation shuffling for 5 features using 891 rows with 3 shuffle sets...
	0.94s	= Expected runtime (0.31s per shuffle set)
	0.29s	= Actual runtime (Completed 3 of 3 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Sex,0.252525,0.013654,0.000487,3,0.330763,0.174288
Age,0.102507,0.007302,0.000844,3,0.144350,0.060663
Pclass,0.101384,0.009545,0.001471,3,0.156080,0.046688
SibSp,0.025065,0.002824,0.002103,3,0.041250,0.008881
Parch,0.016087,0.001296,0.001078,3,0.023513,0.008661


In [13]:
model.get_model_names()

['RandomForestGini',
 'RandomForestEntr',
 'ExtraTreesGini',
 'ExtraTreesEntr',
 'KNeighborsUnif',
 'KNeighborsDist',
 'LightGBM',
 'LightGBMXT',
 'CatBoost',
 'XGBoost',
 'NeuralNetMXNet',
 'LightGBMCustom',
 'WeightedEnsemble_L1']

In [14]:
model.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val  pred_time_val  fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L1   0.849162       0.010180  1.043185                0.001041           0.366748            1       True         13
1               XGBoost   0.843575       0.005028  0.343526                0.005028           0.343526            0       True         10
2        LightGBMCustom   0.837989       0.004111  0.332911                0.004111           0.332911            0       True         12
3              CatBoost   0.826816       0.001593  0.213451                0.001593           0.213451            0       True          9
4              LightGBM   0.826816       0.003844  0.199066                0.003844           0.199066            0       True          7
5        NeuralNetMXNet   0.810056       0.015340  4.264452                0.015340           4.264452        

{'feature_prune': False,
 'hyperparameter_tune': False,
 'hyperparameters_userspecified': {'default': {'CAT': [{}],
   'GBM': [{}, {'AG_args': {'name_suffix': 'XT'}, 'extra_trees': True}],
   'KNN': [{'AG_args': {'name_suffix': 'Unif'}, 'weights': 'uniform'},
    {'AG_args': {'name_suffix': 'Dist'}, 'weights': 'distance'}],
   'NN': [{}],
   'RF': [{'AG_args': {'name_suffix': 'Gini',
      'problem_types': ['binary', 'multiclass']},
     'criterion': 'gini'},
    {'AG_args': {'name_suffix': 'Entr',
      'problem_types': ['binary', 'multiclass']},
     'criterion': 'entropy'}],
   'XGB': [{}],
   'XT': [{'AG_args': {'name_suffix': 'Gini',
      'problem_types': ['binary', 'multiclass']},
     'criterion': 'gini'},
    {'AG_args': {'name_suffix': 'Entr',
      'problem_types': ['binary', 'multiclass']},
     'criterion': 'entropy'}],
   'custom': [{'AG_args': {'disable_in_hpo': True,
      'model_type': 'GBM',
      'name_suffix': 'Custom'},
     'boosting_type': 'gbdt',
     'feature_f

In [15]:
model.info()

{'best_model': 'WeightedEnsemble_L1',
 'best_model_score_val': 0.8491620111731844,
 'best_model_stack_level': 1,
 'eval_metric': 'accuracy',
 'label': 'Survived',
 'max_core_stack_level': 0,
 'max_stack_level': 1,
 'model_info': {'CatBoost': {'AG_args_fit': {'get_features_kwargs': None,
    'get_features_kwargs_extra': None,
    'ignored_type_group_raw': ['object'],
    'ignored_type_group_special': None,
    'max_memory_usage_ratio': 1.0,
    'max_time_limit': None,
    'max_time_limit_ratio': 1.0,
    'min_time_limit': 0},
   'eval_metric': 'accuracy',
   'feature_metadata': <autogluon.tabular.features.feature_metadata.FeatureMetadata at 0x7f8fc1942e48>,
   'features': ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch'],
   'fit_time': 0.21345067024230957,
   'hyperparameters': {'allow_writing_files': False,
    'eval_metric': 'Accuracy',
    'iterations': 10000,
    'learning_rate': 0.1,
    'random_seed': 0},
   'hyperparameters_fit': {'iterations': 17},
   'hyperparameters_nondefault': [],